<a href="https://colab.research.google.com/github/LalchandShivraj/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubu

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [60]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

In [61]:
# create df by filtering the dataset where the total_votes >= 20 pick reviews.
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vote_count_df = vine_df.filter("total_votes >= 20")


In [62]:
# create a new df by filtering the preceding df where helpful_votes/total_votes is >= 50%
votes_df = vote_count_df.filter(vote_count_df["helpful_votes"]/vote_count_df["total_votes"]>=0.5)
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [65]:
# create a new df by filtering where a review was written as part of the Vine program(paid), vine =='Y'
vine_program_df = votes_df.filter(votes_df["vine"] == "Y")

In [64]:
# filter the votes_df for rows not in the Vine Program.
not_vine_program_df = votes_df.filter(votes_df["vine"] == "N")

In [66]:
# Determine the total number of reviews, the number of 5-star reviews, 
# and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

# number of reviews:
votes_df.count()

39095

In [67]:
# number of Vine reviews:
vine_reviews = votes_df.filter(votes_df["vine"]== "Y").count()
vine_reviews

266

In [68]:
# number of non-Vine reviews:
nonvine_reviews = votes_df.filter(votes_df["vine"]== "N").count()
nonvine_reviews

38829

In [69]:
# Vine reviews that were 5-stars
vine_reviews_5stars = votes_df.filter( (votes_df["vine"]== "Y") & (votes_df["star_rating"]== 5) ).count()
vine_reviews_5stars

125

In [70]:
# non-Vine reviews that were 5-stars
nonvine_reviews_5stars = votes_df.filter( (votes_df["vine"]== "N") & (votes_df["star_rating"]== 5) ).count()
nonvine_reviews_5stars

18246

In [72]:
# percentage of Vine reviews that were 5-stars
(vine_reviews_5stars / vine_reviews) * 100

46.99248120300752

In [73]:
# percentage of non-Vine reviews that were 5-stars
(nonvine_reviews_5stars / nonvine_reviews) * 100

46.99065131731438